In [63]:
import pyomo.environ as pyo

def build_model():
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.y1 +1.5*m.y2 + 0.5*m.y3 + m.x1**2 + m.x2**2

    @m.Constraint()
    def g1(m):
        return (m.x1-2)**2 - m.x2 <= 0 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    return m


def solve(m, solver):
    opt = pyo.SolverFactory('gams', solver=solver)
    results = opt.solve(m, tee=False,add_options=['option optcr = 0.0;'])
    return m, results

m = build_model()
m, results = solve(m, 'baron')
print('BARON: | x1 =', pyo.value(m.x1), '  |  x2 =', pyo.value(m.x2), '  |  y1 =', pyo.value(m.y1), '  |  y2 =', pyo.value(m.y2), '  |  y3 =', pyo.value(m.y3), '  |  obj =', pyo.value(m.obj), '  |  status =', results.solver.status)

BARON: | x1 = 1.0   |  x2 = 1.0   |  y1 = 0.0   |  y2 = 1.0   |  y3 = 0.0   |  obj = 3.5   |  status = ok


## NLP Subproblem

In [64]:
def subproblem(y1, y2, y3):
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.y1 +1.5*m.y2 + 0.5*m.y3 + m.x1**2 + m.x2**2

    @m.Constraint()
    def g1(m):
        return (m.x1-2)**2 - m.x2 <= 0 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

    m.y1.fix(y1)
    m.y2.fix(y2)
    m.y3.fix(y3)
    return m

## Master Outer Approximation Problem

In [65]:
def master_oa(xk1, xk2):
    m = pyo.ConcreteModel()

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    m.alpha = pyo.Var(within=pyo.Reals)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha

    @m.Constraint()
    def alpha_con(m):
        return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2)


    @m.Constraint()
    def g1_lin(m):
        return (xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    return m


## Outer Approximation

In [76]:
# Iteration 1
# Solve the NLP subproblem
m_sub = subproblem(1, 1, 1)
m_sub, results = solve(m_sub, 'baron')
print('Outer approximation method')
print('Iteration 1')
print('NLP: | x1 =', pyo.value(m_sub.x1), '  |  x2 =', pyo.value(m_sub.x2), '  |  y1 =', pyo.value(m_sub.y1), '  |  y2 =', pyo.value(m_sub.y2), '  |  y3 =', pyo.value(m_sub.y3), '  |  obj =', pyo.value(m_sub.obj), '  |  status =', results.solver.status)

# Solve the master problem
m_master = master_oa(pyo.value(m_sub.x1), pyo.value(m_sub.x2))
m_master, results = solve(m_master, 'cplex')
print('Master: | x1 =', pyo.value(m_master.x1), '  |  x2 =', pyo.value(m_master.x2), '  |  y1 =', pyo.value(m_master.y1), '  |  y2 =', pyo.value(m_master.y2), '  |  y3 =', pyo.value(m_master.y3), '  |  alpha =', pyo.value(m_master.alpha), '  |  status =', results.solver.status)
print('Upper bound: ', pyo.value(m_sub.obj), 'Lower bound: ', pyo.value(m_master.alpha))
print()

# Iteration 2
# Solve the NLP subproblem
m_sub = subproblem(pyo.value(m_master.y1), pyo.value(m_master.y2), pyo.value(m_master.y3))
m_sub, results = solve(m_sub, 'baron')
print('Iteration 2')
print('NLP: | x1 =', pyo.value(m_sub.x1), '  |  x2 =', pyo.value(m_sub.x2), '  |  y1 =', pyo.value(m_sub.y1), '  |  y2 =', pyo.value(m_sub.y2), '  |  y3 =', pyo.value(m_sub.y3), '  |  obj =', pyo.value(m_sub.obj), '  |  status =', results.solver.status)

# Solve the master problem
m_master = master_oa(pyo.value(m_sub.x1), pyo.value(m_sub.x2))

@m_master.Constraint()
def alpha_con1(m):
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 +k2**2 + 2*k1*(m.x1 - k1) + 2*k2*(m.x2 - k2)

m_master, results = solve(m_master, 'cplex')
print('Master: | x1 =', pyo.value(m_master.x1), '  |  x2 =', pyo.value(m_master.x2), '  |  y1 =', pyo.value(m_master.y1), '  |  y2 =', pyo.value(m_master.y2), '  |  y3 =', pyo.value(m_master.y3), '  |  alpha =', pyo.value(m_master.alpha), '  |  status =', results.solver.status)
print('Upper bound: ', pyo.value(m_sub.obj), 'Lower bound: ', pyo.value(m_master.alpha))
print()

# Iteration 3
# Solve the NLP subproblem
m_sub = subproblem(pyo.value(m_master.y1), pyo.value(m_master.y2), pyo.value(m_master.y3))
m_sub, results = solve(m_sub, 'baron')
print('Iteration 3')
print('NLP: | x1 =', pyo.value(m_sub.x1), '  |  x2 =', pyo.value(m_sub.x2), '  |  y1 =', pyo.value(m_sub.y1), '  |  y2 =', pyo.value(m_sub.y2), '  |  y3 =', pyo.value(m_sub.y3), '  |  obj =', pyo.value(m_sub.obj), '  |  status =', results.solver.status)

# Solve the master problem
m_master = master_oa(pyo.value(m_sub.x1), pyo.value(m_sub.x2))

@m_master.Constraint()
def alpha_con1(m):
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 +k2**2 + 2*k1*(m.x1 - k1) + 2*k2*(m.x2 - k2)

@m_master.Constraint()
def alpha_con2(m):
    k1 = 2
    k2 = 0
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 +k2**2 + 2*k1*(m.x1 - k1) + 2*k2*(m.x2 - k2)

m_master, results = solve(m_master, 'cplex')
print('Master: | x1 =', pyo.value(m_master.x1), '  |  x2 =', pyo.value(m_master.x2), '  |  y1 =', pyo.value(m_master.y1), '  |  y2 =', pyo.value(m_master.y2), '  |  y3 =', pyo.value(m_master.y3), '  |  alpha =', pyo.value(m_master.alpha), '  |  status =', results.solver.status)
print('Upper bound: ', pyo.value(m_sub.obj), 'Lower bound: ', pyo.value(m_master.alpha))
print()

Outer approximation method
Iteration 1
NLP: | x1 = 2.0   |  x2 = 2.0   |  y1 = 1   |  y2 = 1   |  y3 = 1   |  obj = 11.0   |  status = ok
Master: | x1 = 2.0   |  x2 = 0.0   |  y1 = 1.0   |  y2 = 0.0   |  y3 = 0.0   |  alpha = 1.0   |  status = ok
Upper bound:  11.0 Lower bound:  1.0

Iteration 2
NLP: | x1 = 2.0   |  x2 = 0.0   |  y1 = 1.0   |  y2 = 0.0   |  y3 = 0.0   |  obj = 5.0   |  status = ok
Master: | x1 = 1.0   |  x2 = 1.0   |  y1 = 0.0   |  y2 = 1.0   |  y3 = 0.0   |  alpha = 1.5   |  status = ok
Upper bound:  5.0 Lower bound:  1.5

Iteration 3
NLP: | x1 = 1.0   |  x2 = 1.0   |  y1 = 0.0   |  y2 = 1.0   |  y3 = 0.0   |  obj = 3.5   |  status = ok
Master: | x1 = 1.0   |  x2 = 1.0   |  y1 = 0.0   |  y2 = 1.0   |  y3 = 0.0   |  alpha = 3.5   |  status = ok
Upper bound:  3.5 Lower bound:  3.5



## Generalized Benders Decomposition

In [102]:
# Generalized Benders Decomposition
def master_gbd():
    m = pyo.ConcreteModel()

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    m.alpha = pyo.Var(within=pyo.Reals)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha

    return m

# Iteration 1
print('Iteration 1')

# NLP subproblem
m_sub = subproblem(1, 1, 1)
ms, results = solve(m_sub, 'baron')
# ms.dual.pprint()
print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
print('Non-zero multipliers: | mu2:',8,'| mu3:',4)

# Master Benders Problem
m_master = master_gbd()
@m_master.Constraint()
def alpha_it1(m):
    mu2 = 8
    mu3 = 4
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2*(-k1+2*m.y1) + mu3*(k1 - k2 - 4 + 4*m.y2)

m, results = solve(m_master, 'cplex')

print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print('UB:', pyo.value(ms.obj), 'and LB:', pyo.value(m.obj))
print()

# Iteration 2
print('Iteration 2')

# # NLP subproblem
# m_sub = subproblem(pyo.value(m.y1), pyo.value(m.y2), pyo.value(m.y3))
# ms, results = solve(m_sub, 'baron')
# # ms.dual.pprint()
print('NLP Infeasible, New Feasibility Cut.')
print('Non-zero multipliers: | mu7:',1)

# Master Benders Problem
m_master = master_gbd()
@m_master.Constraint()
def alpha_it1(m):
    mu2 = 8
    mu3 = 4
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2*(-k1+2*m.y1) + mu3*(k1 - k2 - 4 + 4*m.y2)

@m_master.Constraint()
def alpha_it2(m):
    mu7 = 1
    return mu7*(1 - m.y1 - m.y2 - m.y3) <= 0 

m, results = solve(m_master, 'cplex')

print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print('UB:', pyo.value(ms.obj), 'and LB:', pyo.value(m.obj))
print()

# Iteration 3
print('Iteration 3')

# NLP subproblem
m_sub = subproblem(pyo.value(m.y1), pyo.value(m.y2), pyo.value(m.y3))
ms, results = solve(m_sub, 'baron')
# ms.dual.pprint()
print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
print('Non-zero multipliers: | mu6:',3)

# Master Benders Problem
m_master = master_gbd()
@m_master.Constraint()
def alpha_it1(m):
    mu2 = 8
    mu3 = 4
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2*(-k1+2*m.y1) + mu3*(k1 - k2 - 4 + 4*m.y2)

@m_master.Constraint()
def alpha_it2(m):
    mu7 = 1
    return mu7*(1 - m.y1 - m.y2 - m.y3) <= 0

@m_master.Constraint()
def alpha_it3(m):
    mu6 = 3
    k1 = 1.5
    k2 = 1.5
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu6*(-k1 -k2 + 3*m.y3)

m, results = solve(m_master, 'cplex')

print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print('UB:', pyo.value(ms.obj), 'and LB:', pyo.value(m.obj))
print()

# Iteration 4
print('Iteration 4')

# NLP subproblem
m_sub = subproblem(pyo.value(m.y1), pyo.value(m.y2), pyo.value(m.y3))
ms, results = solve(m_sub, 'baron')
# ms.dual.pprint()
print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
print('Non-zero multipliers: | mu2:',4)

# Master Benders Problem
m_master = master_gbd()
@m_master.Constraint()
def alpha_it1(m):
    mu2 = 8
    mu3 = 4
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2*(-k1+2*m.y1) + mu3*(k1 - k2 - 4 + 4*m.y2)

@m_master.Constraint()
def alpha_it2(m):
    mu7 = 1
    return mu7*(1 - m.y1 - m.y2 - m.y3) <= 0

@m_master.Constraint()
def alpha_it3(m):
    mu6 = 3
    k1 = 1.5
    k2 = 1.5
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu6*(-k1 -k2 + 3*m.y3)

@m_master.Constraint()
def alpha_it4(m):
    mu2 = 4
    k1 = 2
    k2 = 0
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2 * (-k1 + 2*m.y1)

m, results = solve(m_master, 'cplex')

print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print('UB:', pyo.value(ms.obj), 'and LB:', pyo.value(m.obj))
print()

# Iteration 5
print('Iteration 5')

# NLP subproblem
m_sub = subproblem(pyo.value(m.y1), pyo.value(m.y2), pyo.value(m.y3))
ms, results = solve(m_sub, 'baron')
# ms.dual.pprint()
print('NLP: | x1:',pyo.value(ms.x1),'| x2:',pyo.value(ms.x2), '| y1:',pyo.value(ms.y1), '| y2:',pyo.value(ms.y2), '| y3:',pyo.value(ms.y3),'| Objective:',pyo.value(ms.obj), '| Solver Status.:',results.solver.termination_condition)
print('Non-zero multipliers: | mu1:', 4/3, ' | mu3:', 2/3)

# Master Benders Problem
m_master = master_gbd()
@m_master.Constraint()
def alpha_it1(m):
    mu2 = 8
    mu3 = 4
    k1 = 2
    k2 = 2
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2*(-k1+2*m.y1) + mu3*(k1 - k2 - 4 + 4*m.y2)

@m_master.Constraint()
def alpha_it2(m):
    mu7 = 1
    return mu7*(1 - m.y1 - m.y2 - m.y3) <= 0

@m_master.Constraint()
def alpha_it3(m):
    mu6 = 3
    k1 = 1.5
    k2 = 1.5
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu6*(-k1 -k2 + 3*m.y3)

@m_master.Constraint()
def alpha_it4(m):
    mu2 = 4
    k1 = 2
    k2 = 0
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu2 * (-k1 + 2*m.y1)

@m_master.Constraint()
def alpha_it5(m):
    mu1 = 4/3
    mu3 = 2/3
    k1 = 1
    k2 = 1
    return m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + k1**2 + k2**2 + mu1*((k1-2)**2 - k2) + mu3*(k1 - k2 - 4 + 4*m.y2)

m, results = solve(m_master, 'cplex')

print('Master MILP: | y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print('UB:', pyo.value(ms.obj), 'and LB:', pyo.value(m.obj))

Iteration 1
NLP: | x1: 2.0 | x2: 2.0 | y1: 1 | y2: 1 | y3: 1 | Objective: 11.0 | Solver Status.: optimal
Non-zero multipliers: | mu2: 8 | mu3: 4
Master MILP: | y1: 0.0 | y2: 0.0 | y3: 0.0 | Objective: -24.0 | Solver Status.: optimal
UB: 11.0 and LB: -24.0

Iteration 2
NLP Infeasible, New Feasibility Cut.
Non-zero multipliers: | mu7: 1
Master MILP: | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: -23.5 | Solver Status.: optimal
UB: 11.0 and LB: -23.5

Iteration 3
NLP: | x1: 1.499999999853751 | x2: 1.499999999849582 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 4.999999999109999 | Solver Status.: optimal
Non-zero multipliers: | mu6: 3
Master MILP: | y1: 1.0 | y2: 0.0 | y3: 0.0 | Objective: -3.5 | Solver Status.: optimal
UB: 4.999999999109999 and LB: -3.5

Iteration 4
NLP: | x1: 2.0 | x2: 0.0 | y1: 1.0 | y2: 0.0 | y3: 0.0 | Objective: 5.0 | Solver Status.: optimal
Non-zero multipliers: | mu2: 4
Master MILP: | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: -2.5 | Solver Status.: optimal
UB: 5.0 and LB: -

## Extended Cutting Plane

In [105]:
def master_ecp():
    m = pyo.ConcreteModel()

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.alpha = pyo.Var(within=pyo.Reals)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha
    
    m.alpha_constraint = pyo.ConstraintList()
    m.g1_linearization = pyo.ConstraintList()

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0 

    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0

    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0

    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0

    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0

    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0

    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0 

    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0 

    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0 

    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0

    return m

In [116]:
m = master_ecp()
m.y1.set_value(1)
m.y2.set_value(1)
m.y3.set_value(1)

xk1 = 0
xk2 = 0

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print('ECP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print()

xk1 = 1
xk2 = 4

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print('ECP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print()

xk1 = 3
xk2 = 0

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print('ECP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print()

xk1 = 7/6
xk2 = 11/6

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print('ECP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print()

xk1 = 1
xk2 = 1

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print('ECP: | x1:',pyo.value(m.x1),'| x2:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)
print()

ECP: | x1: 1.0 | x2: 4.0 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal

ECP: | x1: 3.0 | x2: 0.0 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal

ECP: | x1: 1.166666666666667 | x2: 1.833333333333333 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal

ECP: | x1: 1.0 | x2: 1.0 | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 2.777777777777779 | Solver Status.: optimal

ECP: | x1: 1.0 | x2: 1.0 | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 3.5 | Solver Status.: optimal



## Extended Supporting Hyperplane

In [117]:
def master_esh():
    m = pyo.ConcreteModel()

    m.y1 = pyo.Var(within=pyo.Binary)
    m.y2 = pyo.Var(within=pyo.Binary)
    m.y3 = pyo.Var(within=pyo.Binary)

    m.x1 = pyo.Var(within=pyo.Reals)
    m.x2 = pyo.Var(within=pyo.Reals)

    m.alpha = pyo.Var(within=pyo.Reals)

    @m.Objective(sense=pyo.minimize)
    def obj(m):
        return m.alpha
    
    m.alpha_constraint = pyo.ConstraintList()
    m.g1_linearization = pyo.ConstraintList()

    @m.Constraint()
    def g2(m):
        return -m.x1 +2*m.y1 <= 0
    
    @m.Constraint()
    def g3(m):
        return m.x1 - m.x2 - 4 + 4*m.y2 <= 0
    
    @m.Constraint()
    def g4(m):
        return -m.x1 + 1 -m.y1 <= 0
    
    @m.Constraint()
    def g5(m):
        return -m.x2 + m.y2 <= 0
    
    @m.Constraint()
    def g6(m):
        return -m.x1 - m.x2 + 3*m.y3 <= 0
    
    @m.Constraint()
    def g7(m):
        return 1 - m.y1 - m.y2 - m.y3 <= 0
    
    @m.Constraint()
    def g8(m):
        return m.x1 - 4 <= 0
    
    @m.Constraint()
    def g9(m):
        return m.x2 - 4 <= 0
    
    @m.Constraint()
    def g10(m):
        return -m.x1 <= 0
    
    @m.Constraint()
    def g11(m):
        return -m.x2 <= 0
    
    return m



In [122]:
import numpy as np

m = master_esh()
m.y1.set_value(1)
m.y2.set_value(1)
m.y3.set_value(1)

xk1 = 0
xk2 = 0

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print('ESH: | x1_MIP:',pyo.value(m.x1),'| x2_MIP:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)

xk1 = 4 - 2 * np.sqrt(3)
xk2 = 16 - 8 * np.sqrt(3)

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print()
print('ESH: | x1_MIP:',pyo.value(m.x1),'| x2_MIP:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)

xk1 = 2
xk2 = 0

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print()
print('ESH: | x1_MIP:',pyo.value(m.x1),'| x2_MIP:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)

xk1 = 1
xk2 = 1

m.alpha_constraint.add(m.alpha >= m.y1 + 1.5*m.y2 + 0.5*m.y3 + xk1**2 + xk2**2 + 2*xk1*(m.x1 - xk1) + 2*xk2*(m.x2 - xk2))
m.g1_linearization.add((xk1-2)**2 - xk2 + 2*(xk1-2)*(m.x1 - xk1) - (m.x2 - xk2) <= 0 )

m, results = solve(m, 'cplex')

print()
print('ESH: | x1_MIP:',pyo.value(m.x1),'| x2_MIP:',pyo.value(m.x2), '| y1:',pyo.value(m.y1), '| y2:',pyo.value(m.y2), '| y3:',pyo.value(m.y3),'| Objective:',pyo.value(m.obj), '| Solver Status.:',results.solver.termination_condition)

ESH: | x1_MIP: 1.0 | x2_MIP: 4.0 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal

ESH: | x1_MIP: 3.0 | x2_MIP: 0.0 | y1: 0.0 | y2: 0.0 | y3: 1.0 | Objective: 0.5 | Solver Status.: optimal

ESH: | x1_MIP: 1.0 | x2_MIP: 1.0 | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 1.976803507357076 | Solver Status.: optimal

ESH: | x1_MIP: 1.0 | x2_MIP: 1.0 | y1: 0.0 | y2: 1.0 | y3: 0.0 | Objective: 3.5 | Solver Status.: optimal
